<a href="https://colab.research.google.com/github/afeng9-999/F25-STATS101C-FinalProject/blob/main/madelyn_stats_101c_final_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Splines - 0.446997

In [ ]:
import math
import csv
import pandas as pd
import numpy as np

from collections import Counter
import sklearn

from sklearn.preprocessing import LabelEncoder, SplineTransformer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import roc_auc_score, log_loss

ModuleNotFoundError: No module named 'catboost'

In [ ]:
train = pd.read_csv("aluminum_coldRoll_train.csv")
final_test = pd.read_csv("aluminum_coldRoll_testNoY.csv")

In [ ]:
cat_cols = train.select_dtypes(include=['object']).columns.tolist()
label_encoders = {}
for col in cat_cols:
    le = LabelEncoder()
    le.fit(train[col])
    train[col] = le.transform(train[col])
    final_test[col] = final_test[col].apply(lambda x: x if x in le.classes_ else "UNSEEN")
    le.classes_ = np.append(le.classes_, "UNSEEN")
    final_test[col] = le.transform(final_test[col])
    label_encoders[col] = le

In [ ]:
train.head()

,ID,alloy,cutTemp,rollTemp,firstPassRollPressure,secondPassRollPressure,topEdgeMicroChipping,blockSource,machineRestart,contourDefNdx,clearPassNdx,y_passXtremeDurability
0,1,2,2,2,500,350,0,2,0,5,2.013540,0
1,2,3,1,0,575,325,0,0,0,2,1.887291,1
2,3,6,2,1,450,350,0,1,0,2,2.016347,1
3,4,2,0,1,600,500,0,1,0,5,1.887250,0
4,5,5,1,1,525,425,0,0,0,5,2.156250,0


In [ ]:
target = "y_passXtremeDurability"
X = train.drop(columns = [target])
y = train[target]
num_cols = X.select_dtypes(include=[np.number]).columns.tolist()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 42)

In [ ]:
preprocess = ColumnTransformer(
    transformers=[
        ("spline", SplineTransformer(degree=3, n_knots=5), num_cols)
    ],
    remainder="passthrough"
)

In [ ]:
model = Pipeline([
    ("prep", preprocess),
    ("clf", GradientBoostingClassifier())
])
model.fit(X_train, y_train)

KeyboardInterrupt: 

In [ ]:
val_pred = model.predict_proba(X_test)[:, 1]
print("AUC :", roc_auc_score(y_test, val_pred))
print("LogLoss :", log_loss(y_test, val_pred))

AUC : 0.8725761625263426
LogLoss : 0.44771581946726124


In [ ]:
test_pred = model.predict_proba(final_test)[:, 1]

In [ ]:
data = pd.DataFrame({
    "ID": final_test["ID"],
    "y_passXtremeDurability": test_pred
})
data.to_csv("spline_submission1.csv", index = False)
from google.colab import files
files.download("spline_submission1.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### CatBoost - 0.427229

In [ ]:
!pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 8.1 MB/s eta 0:00:00


In [ ]:
## what I used earlier (catboost 2)
model = CatBoostClassifier(
   model = CatBoostClassifier(
    loss_function="Logloss",
    eval_metric="Logloss",
    learning_rate=0.05,
    depth=6,
    iterations=600,
    l2_leaf_reg=3,
    random_seed=42,
    verbose=200
)

In [ ]:
from catboost import CatBoostClassifier, Pool

train = pd.read_csv("aluminum_coldRoll_train.csv")
test = pd.read_csv("aluminum_coldRoll_testNoY.csv")

target = "y_passXtremeDurability"
X = train.drop(columns=[target])
y = train[target]

cat_cols = X.select_dtypes(include=["object"]).columns.to_list()

X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

train_pool = Pool(X_train, y_train, cat_features=cat_cols)
val_pool   = Pool(X_val, y_val, cat_features=cat_cols)
test_pool  = Pool(test, cat_features=cat_cols)

#catboost 3
model = CatBoostClassifier(
    loss_function="Logloss",
    eval_metric="Logloss",
    learning_rate=0.07,
    depth=5,
    iterations=800,
    random_seed=42,
    verbose=200,
    subsample=0.9,
    colsample_bylevel=0.9,
    l2_leaf_reg = 6,
    od_type="Iter",
    od_wait=100,
)

model.fit(train_pool, eval_set=val_pool, use_best_model=True)
val_pred = model.predict_proba(val_pool)[:, 1]
val_logloss = log_loss(y_val, val_pred)
val_auc = roc_auc_score(y_val, val_pred)


0:	learn: 0.6646276	test: 0.6648063	best: 0.6648063 (0)	total: 1.2s	remaining: 15m 55s
200:	learn: 0.4246990	test: 0.4261056	best: 0.4261056 (200)	total: 41.9s	remaining: 2m 5s
400:	learn: 0.4206617	test: 0.4242736	best: 0.4242736 (400)	total: 1m 12s	remaining: 1m 12s
600:	learn: 0.4183189	test: 0.4240956	best: 0.4240482 (540)	total: 1m 46s	remaining: 35.1s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 0.4240481713
bestIteration = 540

Shrink model to first 541 iterations.


In [ ]:
print(f"Validation LogLoss: {val_logloss:.4f}")
print(f"Validation ROC AUC: {val_auc:.4f}")

Validation LogLoss: 0.4240
Validation ROC AUC: 0.8839


In [ ]:
test_pred = model.predict_proba(test_pool)[:, 1]
test_pred = np.clip(test_pred, 1e-15, 1-1e-15)

In [ ]:
data2 = pd.DataFrame({
    "ID": test["ID"],
    "y_passXtremeDurability": test_pred
})
from google.colab import files
data2.to_csv("catboost3.csv", index = False)
files.download("catboost3.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### XGBoost (don't use this)

In [ ]:
!pip install --upgrade xgboost

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import log_loss, roc_auc_score
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder

In [ ]:
train = pd.read_csv("aluminum_coldRoll_train.csv")
test = pd.read_csv("aluminum_coldRoll_testNoY.csv")
target = "y_passXtremeDurability"
X = train.drop(columns=[target])
y = train[target]
cat_cols = X.select_dtypes(include=["object"]).columns.to_list()
for col in cat_cols:
    le = LabelEncoder()
    X[col] = le.fit_transform(X[col])
    test[col] = le.transform(test[col].astype(str))
n_splits = 5
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

In [ ]:
oof_pred_xgb = np.zeros(len(X))
test_pred_xgb = np.zeros(len(test))

for fold, (tr_idx, val_idx) in enumerate(skf.split(X, y), 1):
    print(f"\n===== Fold {fold} =====")
    X_tr, X_val = X.iloc[tr_idx], X.iloc[val_idx]
    y_tr, y_val = y.iloc[tr_idx], y.iloc[val_idx]

    model = xgb.XGBClassifier(
        n_estimators=150,
        learning_rate=0.1,
        max_depth=6,
        objective='binary:logistic',
        eval_metric='logloss'
    )

    model.fit(X_tr, y_tr, eval_set=[(X_val, y_val)], verbose=True)

    val_pred = model.predict_proba(X_val)[:, 1]
    oof_pred_xgb[val_idx] = val_pred
    test_pred_xgb += model.predict_proba(test)[:, 1] / n_splits


===== Fold 1 =====
[0]	validation_0-logloss:0.65826
[1]	validation_0-logloss:0.63330
[2]	validation_0-logloss:0.61222
[3]	validation_0-logloss:0.59444
[4]	validation_0-logloss:0.57973
[5]	validation_0-logloss:0.56662
[6]	validation_0-logloss:0.55576
[7]	validation_0-logloss:0.54576
[8]	validation_0-logloss:0.53744
[9]	validation_0-logloss:0.53018
[10]	validation_0-logloss:0.52367
[11]	validation_0-logloss:0.51758
[12]	validation_0-logloss:0.51258
[13]	validation_0-logloss:0.50798
[14]	validation_0-logloss:0.50400
[15]	validation_0-logloss:0.49982
[16]	validation_0-logloss:0.49612
[17]	validation_0-logloss:0.49229
[18]	validation_0-logloss:0.48863
[19]	validation_0-logloss:0.48555
[20]	validation_0-logloss:0.48335
[21]	validation_0-logloss:0.48136
[22]	validation_0-logloss:0.47713
[23]	validation_0-logloss:0.47535
[24]	validation_0-logloss:0.47343
[25]	validation_0-logloss:0.46968
[26]	validation_0-logloss:0.46820
[27]	validation_0-logloss:0.46674
[28]	validation_0-logloss:0.46366
[29]

In [ ]:
def clip(p):
    return np.clip(p, 1e-6, 1-1e-6)

cv_logloss = log_loss(y, clip(oof_pred_xgb))
cv_auc = roc_auc_score(y, oof_pred_xgb)
print(f"\n5-fold OOF LogLoss: {cv_logloss:.6f}")
print(f"5-fold OOF AUC: {cv_auc:.6f}")


5-fold OOF LogLoss: 0.427291
5-fold OOF AUC: 0.882642


### Catboost improved with cv (built off of Luis's)

In [ ]:
!pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 7.9 MB/s eta 0:00:00


In [ ]:
from catboost import CatBoostClassifier, Pool
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import log_loss, roc_auc_score
import numpy as np
import pandas as pd

train = pd.read_csv("aluminum_coldRoll_train.csv")
test = pd.read_csv("aluminum_coldRoll_testNoY.csv")

target = "y_passXtremeDurability"
X = train.drop(columns=[target])
y = train[target]

cat_cols = X.select_dtypes(include=["object"]).columns.to_list()

In [ ]:
# cross validation
n_splits = 5
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

oof_pred = np.zeros(len(train))
test_pred = np.zeros(len(test))
best_its = []

In [ ]:
for fold, (tr_idx, val_idx) in enumerate(skf.split(X, y), 1):
    print(f"\n===== Fold {fold} =====")
    X_tr, X_val = X.iloc[tr_idx], X.iloc[val_idx]
    y_tr, y_val = y.iloc[tr_idx], y.iloc[val_idx]

    train_pool = Pool(X_tr, y_tr, cat_features=cat_cols)
    val_pool   = Pool(X_val, y_val, cat_features=cat_cols)
    test_pool  = Pool(test, cat_features=cat_cols)

    model = CatBoostClassifier(
        loss_function="Logloss",
        eval_metric="Logloss",
        learning_rate=0.05,
        depth=5,
        iterations=1200,
        l2_leaf_reg=6,
        bagging_temperature=0.5,
        od_type="Iter",
        od_wait=100,
        random_seed=42 + fold,
        verbose=200
    )
    model.fit(train_pool, eval_set=val_pool, use_best_model=True)
    best_its.append(model.tree_count_)
    val_pred = model.predict_proba(val_pool)[:, 1]
    oof_pred[val_idx] = val_pred
    test_pred += model.predict_proba(test_pool)[:, 1] / n_splits


===== Fold 1 =====
0:	learn: 0.6742741	test: 0.6741588	best: 0.6741588 (0)	total: 255ms	remaining: 5m 5s
200:	learn: 0.4290216	test: 0.4292532	best: 0.4292532 (200)	total: 44.2s	remaining: 3m 39s
400:	learn: 0.4224410	test: 0.4245986	best: 0.4245875 (396)	total: 1m 25s	remaining: 2m 49s
600:	learn: 0.4203662	test: 0.4240500	best: 0.4240500 (600)	total: 2m 5s	remaining: 2m 5s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 0.4239632747
bestIteration = 683

Shrink model to first 684 iterations.

===== Fold 2 =====
0:	learn: 0.6723926	test: 0.6724858	best: 0.6724858 (0)	total: 220ms	remaining: 4m 23s
200:	learn: 0.4282066	test: 0.4304851	best: 0.4304851 (200)	total: 39.7s	remaining: 3m 17s
400:	learn: 0.4218618	test: 0.4261354	best: 0.4261354 (400)	total: 1m 19s	remaining: 2m 38s
600:	learn: 0.4198945	test: 0.4258115	best: 0.4258104 (599)	total: 2m	remaining: 2m
800:	learn: 0.4182837	test: 0.4257276	best: 0.4257276 (800)	total: 2m 43s	remaining: 1m 21s
Stopped by overf

In [ ]:
test_pred = np.clip(test_pred, 1e-6, 1 - 1e-6)
oof_pred  = np.clip(oof_pred, 1e-6, 1 - 1e-6)
cv_logloss = log_loss(y, oof_pred)
cv_auc = roc_auc_score(y, oof_pred)

In [ ]:
print(f"5-fold OOF LogLoss: {cv_logloss:.6f}")
print(f"5-fold OOF AUC: {cv_auc:.6f}")
print(f"Best iterations per fold: {best_its}")

5-fold OOF LogLoss: 0.423731
5-fold OOF AUC: 0.884638
Best iterations per fold: [684, 858, 707, 750, 795]


In [ ]:
data = pd.DataFrame({
    "ID": test["ID"],
    "y_passXtremeDurability": test_pred
})
data.to_csv("catboost4.csv", index=False)
from google.colab import files
data.to_csv("catboost4.csv", index = False)
files.download("catboost4.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>